Se  tiene  información  estadística  de  la  temporada  regular  de  todos  los jugadores   de   la   NBA   en   un   RDD   de   tuplas   con   el   siguiente   formato:  
`(id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos,  promedio_rebotes,  promedio_faltas). 
`

Un  analista  de  la cadena ESPN  está  trabajando  con  un  RDD que corresponde  a  la primera  ronda  de playoffs y que tiene el siguiente formato:  
`
(id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias,   cantidad_faltas).
`

En   base   a estos   RDDs   se   quiere  programar  en  PySpark  un  programa  que  genere  un  RDD  con  los  nombres  (sin duplicados)  de  los  jugadores  que  lograron  en  algún  partido  de  playoffs  una  cantidad de asistencias mayor a su promedio histórico (el de la temporada regular).  
[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqsptt94yl)

### Llamaremos:

rdd_po: RDD con los datos de los playoffs. <br />
rdd_tr: RDD con los datos de temporada regular.

### Codigo:

In [2]:
# Por si hay mas de un contexto de PySpark corriendo (por ejemplo, otro Notebook), esto para utilizar el mismo.
sc = SparkContext.getOrCreate()

In [3]:
datos_jugadores_regtemp = [
    (1, 'Manu Ginobili', 30, 35, 8, 8, 5, 2.2),
    (2, 'Kobe Bryant', 40, 30, 3, 5, 20, 2.1),
    (3, 'LeBron James', 58, 20, 3, 8, 30, 2.9),
    (4, 'Andre Iguodala', 35, 15, 5, 10, 20, 4),
    (5, 'Carmelo Anthony', 40, 10, 6, 6, 22, 1.1)
]

datos_jugadores_po = [
    (1, 1, 1, 8, 3, 4, 2, 12, 1),
    (1, 1, 3, 4, 1, 5, 2, 15, 1),
    (1, 1, 4, 12, 2, 3, 2, 10, 1),
    (2, 1, 1, 18, 3, 3, 3, 8, 2),
    (2, 1, 4, 9, 3, 4, 1, 5, 1),
    (3, 2, 1, 12, 6, 4, 1, 3, 1),
    (3, 2, 2, 15, 8, 5, 3, 7, 1),
    (3, 2, 3, 22, 6, 2, 1, 4, 2),
    (3, 2, 4, 18, 10, 4, 2, 4, 1),
    (4, 2, 1, 12, 2, 1, 1, 9, 0),
    (4, 2, 2, 25, 8, 4, 2, 4, 3),
    (4, 2, 4, 15, 4, 2, 2, 8, 1),
    (1, 3, 3, 2, 1, 0, 0, 3, 2),
    (5, 3, 1, 8, 1, 2, 0, 6, 0),
    (5, 3, 1, 7, 0, 0, 1, 6, 1),
]


# Los paso a RDDs
rdd_tr = sc.parallelize(datos_jugadores_regtemp)
rdd_po = sc.parallelize(datos_jugadores_po)

### Paso 1

Como primer paso, descarto toda la informacion que no me interesa.

Dejo al RDD de playoffs solo con tuplas de formato: <span style="color:darkred"><em> ((id_jugador, id_partido), cantidad_asistencias) </em></span> <br/>
Dejo al RDD de temporadas regulares solo con tuplas de formato: <span style="color:darkred"><em> (id_jugador, [nombre, promedio_asistencias]) </em></span>

In [4]:
rdd_po = rdd_po.map(lambda x: ( (x[0],x[1]), x[7] ))
rdd_tr = rdd_tr.map(lambda x: ( x[0], [x[1], x[3]] ))

### Paso 2

En este paso, se toma el RDD de playoffs.<br/>
Primero, se agrupan y suman todos los asistencias que corresponden a un mismo jugador en un mismo partido 
(ya que un jugador pudo haber salido y vuelto a entrar, por ejemplo).

Inmediatamente despues, convertimos todos esos registros en tuplas de formato: <br/>
<span style="color:darkred"><em> (id_jugador, cantidad_asistencias_total_en_partido) </em></span> <br/>

In [5]:
rdd_po = rdd_po.reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0], x[1]))

#Hacemos collect y podemos ver como quedo la suma de asistencias de cada jugador en cada partido de playoffs.
rdd_po.collect()

[(4, 21), (1, 37), (1, 3), (5, 12), (3, 18), (2, 13)]

### Paso 3

Ahora, nos quedamos con el maximo de asistencias de cada jugador en un partido de playoffs (si el maximo no supera el promedio, entonces ese jugador no cumple).

Luego, se toma eso y se convierte en tuplas de formato: <br/>
<span style="color:darkred"><em> (id_jugador, [cantidad_asistencias_maximo]) </em></span> <br/>

In [6]:
rdd_po = rdd_po.reduceByKey(lambda x,y: max(x,y)).map(lambda x: (x[0], [x[1]]))

# Hacemos collect y vemos efectivamente que solo quedo la mayor cantidad de asistencias de cada jugador, con ese formato.
rdd_po.collect()

[(4, [21]), (1, [37]), (5, [12]), (2, [13]), (3, [18])]

### Paso 4

Joineamos por id_jugador ambos RDD, por lo que nos quedarian tuplas de formato:<br/>
<span style="color:darkred"><em> (id_jugador, ([cantidad_asistencias_maximo], [nombre, promedio_asistencias])) </em></span> <br/>

Luego, pasa a convertir el segundo elemento de cada registro en una unica lista concatenada de forma: <br/>
<span style="color:darkorange"><em> [cantidad_asistencias_maximo, nombre, promedio_asistencias] </em></span> <br/>

In [7]:
rdd_combinado = rdd_po.join(rdd_tr).map(lambda x: (x[0], x[1][0] + x[1][1]))

# Luego de esos, con el resultado de collect podemos corroborar que quede de dicha forma.
rdd_combinado.collect()

[(1, [37, 'Manu Ginobili', 35]),
 (2, [13, 'Kobe Bryant', 30]),
 (3, [18, 'LeBron James', 20]),
 (4, [21, 'Andre Iguodala', 15]),
 (5, [12, 'Carmelo Anthony', 10])]

### Paso 5

A lo ultimo, filtramos y nos quedamos solo con aquellos que su cantidad_asistencias_maximo es mayor a su promedio_asistencias.<br/>
Paso siguiente, de aquellos jugadores que cumplieron con lo buscado, nos quedamos solo con su nombre.

Devolvemos la lista de nombres de jugadores que superaron su promedio de asistencias en algun partido de playoffs.

In [8]:
rdd_combinado = rdd_combinado.filter(lambda x: x[1][0] > x[1][2]).map(lambda x: x[1][1])
rdd_combinado.collect()

['Manu Ginobili', 'Andre Iguodala', 'Carmelo Anthony']